In [ ]:
pip install -U scikit-learn

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit

In [3]:
data_dir  = "../dataset"

In [4]:
data_list = glob.glob(os.path.join(data_dir, '**.csv'))
data_list

['../dataset\\x_test.csv',
 '../dataset\\x_train.csv',
 '../dataset\\y_train.csv']

In [5]:
id_col = 'ID'
target_col = "target"

In [6]:
y_train = pd.read_csv("%s/y_train.csv" % data_dir, sep=",")
x_train= pd.read_csv("%s/x_train.csv" % data_dir, sep=",")
x_test=pd.read_csv("%s/x_test.csv" % data_dir, sep=",")

In [ ]:
train_df  = y_train.merge(x_train, on="ID")

In [ ]:
train_df.head()

In [7]:
return_cols = [c for c in train_df.columns if c.startswith("abs_ret")]
volume_cols = [c for c in train_df.columns if c.startswith("rel_vol")]
date_col = ["day"]
prod_id_col=["pid"]
other_cols = ["LS" , "NLV"]

NameError: name 'train_df' is not defined

In [ ]:
train_df.describe()

In [ ]:
x_test

In [ ]:
# Let's analyze stock number 1
stock_1 = train_df.loc[train_df['pid'] == 1]
stock_1

In [ ]:
stock_1.info()

In [ ]:
#stock_1.loc[:, 'day'] = pd.to_datetime(stock_1.loc[:,'day'], format="%Y/%m/%d")

In [ ]:
# Simple plotting of Stock1
# First Subplot
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5))
ax1.plot(stock_1["day"], stock_1["abs_ret0"])
ax1.set_xlabel("Date", fontsize=12)
ax1.set_ylabel("Abs_ret")
ax1.set_title("Stock1 Abs_return Var 0 & Abs-Vol over days")
# Second Subplot
ax1.plot(stock_1["day"], stock_1["rel_vol0"], color="green")
ax1.set_xlabel("Date", fontsize=12)
ax1.set_ylabel("Abs_Vol0")
ax1.set_title("Stock1 Abs_return Var 0 & Abs-Vol over days")

# Fourth Subplot
ax2.plot(stock_1["abs_ret0"], stock_1["day"], color="orange")
ax2.set_xlabel("Date", fontsize=12)
ax2.set_ylabel("Stock Price")
ax2.set_title("Stock1 Abs_vol Var 0 over days")
plt.show()


Var during one day of return & Vol

2 sol : index with time vector or long format

In [ ]:
l = [i for i in range(60)]

In [ ]:
# Let's analyze stock number 1
stock_1_day1 = stock_1.loc[stock_1['day'] == 0]
stock_1_day1

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5))
ax1.plot(l, stock_1["abs_ret0"])
ax1.set_xlabel("Date", fontsize=12)
ax1.set_ylabel("Abs_ret")
ax1.set_title("Stock1 Abs_return Var 0 & Abs-Vol over days")

Quick strating kit

In [ ]:
train_df.isnull().sum(axis=1)

In [ ]:
x_test.isnull().sum(axis=1)

Interpoloate nan

In [ ]:
#x_train.iloc[:,3:] = x_train.iloc[:,3:].interpolate(axis=1)
#x_test.iloc[:,3:]  = x_test.iloc[:,3:].interpolate(axis=1)

In [ ]:
x_test.isnull().sum(axis=1)

In [ ]:
np.any(np.isnan(x_train))

In [ ]:
np.all(np.isfinite(x_train))

In [8]:
x_train.fillna(0, inplace=True) 
x_test.fillna(0, inplace=True)

## Feature engineering

Create basic feature

In [9]:
x_train['min_ret']    = np.min(x_train.iloc[:,3:63], axis=1)
x_train['max_ret']    = np.max(x_train.iloc[:,3:63], axis=1)
x_train['std_ret']    = np.std(x_train.iloc[:,3:63], axis=1)
x_train['median_ret'] = np.median(x_train.iloc[:,3:63], axis=1)

In [10]:
x_test['min_ret']    = np.min(x_test.iloc[:,3:63], axis=1)
x_test['max_ret']    = np.max(x_test.iloc[:,3:63], axis=1)
x_test['std_ret']    = np.std(x_test.iloc[:,3:63], axis=1)
x_test['median_ret'] = np.median(x_test.iloc[:,3:63], axis=1)

In [11]:
x_train['min_vol']    = np.min(x_train.iloc[:,64:125], axis=1)
x_train['max_vol']    = np.max(x_train.iloc[:,64:125], axis=1)
x_train['std_vol']    = np.std(x_train.iloc[:,64:125], axis=1)
x_train['median_vol'] = np.median(x_train.iloc[:,64:125], axis=1)

In [12]:
x_test['min_vol']    = np.min(x_test.iloc[:,64:125], axis=1)
x_test['max_vol']    = np.max(x_test.iloc[:,64:125], axis=1)
x_test['std_vol']    = np.std(x_test.iloc[:,64:125], axis=1)
x_test['median_vol'] = np.median(x_test.iloc[:,64:125], axis=1)

Validation

In [13]:
def Error(y_true, y_pred): 
    return np.square(y_true - y_pred)

In [14]:
def MSE(y_true, y_pred): 
    return np.square(y_true - y_pred)

In [15]:
from sklearn.model_selection import train_test_split

train_df  = y_train.merge(x_train, on="ID")
train_df.drop(['ID', 'pid'], axis=1, inplace=True)

train_X_, test_X_, train_y_, test_y_ = train_test_split(train_df.iloc[:,1:], train_df['target'], test_size=0.2, random_state=42)

In [ ]:
!pip install delayed

In [16]:
import delayed

In [17]:
from sklearn.linear_model import LinearRegression

regrLinWeighted = LinearRegression()
regrLinWeighted.fit(train_X_, train_y_)

LinearRegression()

In [18]:
test_X_['predict'] = regrLinWeighted.predict(test_X_)

In [19]:
from sklearn.metrics import mean_squared_error

In [ ]:
(mean_squared_error(test_y_, predictions))

In [20]:
test_y_=pd.DataFrame(test_y_)

In [21]:
test_X_['error']=test_y_["target"] - test_X_['predict']

In [ ]:
test_X_

In [22]:
test_X_.iloc[:,0:134]

,day,abs_ret0,abs_ret1,abs_ret2,abs_ret3,abs_ret4,abs_ret5,abs_ret6,abs_ret7,abs_ret8,...,NLV,min_ret,max_ret,std_ret,median_ret,min_vol,max_vol,std_vol,median_vol,predict
99589,41,0.000000,0.000000,0.327869,0.000000,0.000000,0.000000,0.000000,0.072464,0.018067,...,-1.453430,0.0,0.327869,0.061653,0.000000,0.000000,0.095894,0.020351,0.007550,-1.859037
535740,587,0.479781,0.205128,0.000000,0.306748,0.119760,0.034165,0.136565,0.034072,0.068236,...,-0.430984,0.0,0.479781,0.094186,0.068143,0.005014,0.030123,0.006028,0.015041,-1.276157
228479,230,0.709849,0.719741,0.590842,0.000000,0.587027,0.118099,0.118343,0.029595,0.265487,...,-0.133514,0.0,0.719741,0.162341,0.089339,0.001066,0.070583,0.010798,0.013650,-1.999648
214338,396,0.606290,0.151573,0.037836,0.113550,0.037893,0.151400,0.113723,0.113636,0.304183,...,-1.226465,0.0,0.606290,0.116823,0.075873,0.000000,0.059053,0.013184,0.012316,-1.750885
242520,302,0.680760,0.036088,0.144456,0.108186,0.036023,0.072072,0.000000,0.216216,0.251527,...,0.166484,0.0,0.680760,0.099468,0.071416,0.003962,0.064501,0.012081,0.011202,-2.089311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409873,494,0.042680,0.288215,0.203797,0.042740,0.053430,0.032134,0.053539,0.064212,0.032086,...,0.072834,0.0,0.299369,0.064919,0.043201,0.001552,0.072420,0.013117,0.013906,-1.614390
675991,720,0.643777,0.086393,0.259403,0.172488,0.086096,0.301984,0.086543,0.216450,0.217014,...,-1.119516,0.0,0.643777,0.142153,0.131781,0.000000,0.203502,0.028475,0.008158,-1.337233
5086,54,0.163979,0.164249,0.027518,0.055036,0.082667,0.054945,0.054840,0.054840,0.054855,...,0.378673,0.0,0.164249,0.035694,0.027304,0.002358,0.071608,0.012897,0.012157,-1.971190
146514,378,0.480293,0.037936,0.030326,0.128798,0.030246,0.801573,0.180126,0.086460,0.334851,...,1.747052,0.0,0.801573,0.144624,0.113016,0.005106,0.051718,0.008015,0.014423,-2.289628


In [23]:
test_X_reset = test_X_.copy()

In [24]:
test_X_reset.reset_index(drop=True, inplace=True)
test_X_reset

,day,abs_ret0,abs_ret1,abs_ret2,abs_ret3,abs_ret4,abs_ret5,abs_ret6,abs_ret7,abs_ret8,...,min_ret,max_ret,std_ret,median_ret,min_vol,max_vol,std_vol,median_vol,predict,error
0,41,0.000000,0.000000,0.327869,0.000000,0.000000,0.000000,0.000000,0.072464,0.018067,...,0.0,0.327869,0.061653,0.000000,0.000000,0.095894,0.020351,0.007550,-1.859037,0.498510
1,587,0.479781,0.205128,0.000000,0.306748,0.119760,0.034165,0.136565,0.034072,0.068236,...,0.0,0.479781,0.094186,0.068143,0.005014,0.030123,0.006028,0.015041,-1.276157,0.196941
2,230,0.709849,0.719741,0.590842,0.000000,0.587027,0.118099,0.118343,0.029595,0.265487,...,0.0,0.719741,0.162341,0.089339,0.001066,0.070583,0.010798,0.013650,-1.999648,-0.336535
3,396,0.606290,0.151573,0.037836,0.113550,0.037893,0.151400,0.113723,0.113636,0.304183,...,0.0,0.606290,0.116823,0.075873,0.000000,0.059053,0.013184,0.012316,-1.750885,1.911004
4,302,0.680760,0.036088,0.144456,0.108186,0.036023,0.072072,0.000000,0.216216,0.251527,...,0.0,0.680760,0.099468,0.071416,0.003962,0.064501,0.012081,0.011202,-2.089311,-0.393732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136892,494,0.042680,0.288215,0.203797,0.042740,0.053430,0.032134,0.053539,0.064212,0.032086,...,0.0,0.299369,0.064919,0.043201,0.001552,0.072420,0.013117,0.013906,-1.614390,-0.350081
136893,720,0.643777,0.086393,0.259403,0.172488,0.086096,0.301984,0.086543,0.216450,0.217014,...,0.0,0.643777,0.142153,0.131781,0.000000,0.203502,0.028475,0.008158,-1.337233,-1.358559
136894,54,0.163979,0.164249,0.027518,0.055036,0.082667,0.054945,0.054840,0.054840,0.054855,...,0.0,0.164249,0.035694,0.027304,0.002358,0.071608,0.012897,0.012157,-1.971190,0.779151
136895,378,0.480293,0.037936,0.030326,0.128798,0.030246,0.801573,0.180126,0.086460,0.334851,...,0.0,0.801573,0.144624,0.113016,0.005106,0.051718,0.008015,0.014423,-2.289628,-0.080319


In [25]:
date_series = test_X_reset['day']

In [26]:
X_tree=test_X_reset.iloc[:,1:134]
y_tree = test_X_reset.iloc[:,134:135]

In [ ]:
#y_tree['day']=test_X_.iloc[:,0]

Use train_time_split

In [27]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer

In [28]:
min_date = date_series.min()
max_date = date_series.max()

In [29]:
dates = list(range(min_date, max_date + 1))

In [38]:
split_num = 5
date_gap_num = 1

splitter = TimeSeriesSplit(n_splits=split_num,gap=date_gap_num)

date_splits = list(splitter.split(dates))
for i, (train_dates, valid_dates) in enumerate(date_splits):
    print("Date split #{}\n train: {}\nvalid: {}\n\n".format(i, train_dates, valid_dates))

Date split #0
 train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133]
valid: [135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 

In [39]:
def get_indices_from_dates(dates):
    return [ i - 1 for i in date_series[date_series.isin(dates)].index]
split_indices = list()
for train_dates, valid_dates in date_splits:
    train_indices = get_indices_from_dates(train_dates)
    valid_indices = get_indices_from_dates(valid_dates)
    split_indices.append((train_indices, valid_indices))

In [ ]:
!pip install xgboost

In [43]:
!pip install lightgbm

In [58]:
import xgboost

In [62]:
from sklearn.ensemble import GradientBoostingRegressor

In [46]:
from lightgbm import LGBMClassifier

In [63]:
from xgboost import XGBClassifier

# Replace with your model here, this is just a non-optimized example, with few estimators for fast run-time
#model = XGBClassifier(tree_method='gpu_hist', n_estimators=3, gpu_id=0, n_gpus = 1, random_state = 55, n_jobs = -1)

#model = LGBMClassifier()
model = GradientBoostingRegressor()

In [ ]:
X_tree.values

In [ ]:
y_tree

In [51]:
from sklearn.model_selection import cross_val_score 

In [64]:
cv_scores = cross_val_score(model, X_tree.values, y_tree, cv=split_indices, scoring='neg_mean_squared_error')
print("Scores:", cv_scores, "\tmean score:", np.mean(cv_scores))

C:\Users\antoi\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\antoi\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\antoi\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\antoi\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

Scores: [-0.58634074 -0.55388423 -0.5768915  -0.574442   -0.57826998] 	mean score: -0.5739656888697905


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


tscv = TimeSeriesSplit(n_splits=2)
score_models = {'MSE': make_scorer(mean_squared_error)}
#score_models = {'lift_scorer' : lift_scorer, 'AUC' : 'roc_auc'}

xgb_model = XGBClassifier()
test_params = { 'max_depth': [1,2,4,6],
                    'learning_rate': [0.1,0.2,0.3,0.4],
                    'n_estimators': [20,40,80,120],
                    'min_child_weight' : [3,6,9,12]}
gsearch = GridSearchCV(estimator=xgb_model,n_jobs=-1, scoring=score_models, cv=tscv, verbose=1,
                           return_train_score=False,  param_grid=test_params,refit='MSE')
gsearch.fit(X_tree,y_tree)
result_df = pd.DataFrame(gsearch.cv_results_)
result_df.sort_values('MSE',ascending=False,inplace=True)
result_df['n_weights']= test_n_weights(X,y,test_params)
result_df.to_csv(data_set_path.replace('.csv', '_crossval.csv'), index=False)
plot_regression_test(data_set_path,result_df) 

In [ ]:
dans train train
column prediction
column error


Prediction

In [ ]:
from sklearn.linear_model import LinearRegression

regrLinWeighted = LinearRegression()
regrLinWeighted.fit(train_df.iloc[:,1:], train_df['target'])

x_test['predict'] = regrLinWeighted.predict(x_test.iloc[:,3:])

In [ ]:
predictions = x_test[['ID', 'predict']]
predictions.columns = ['ID', 'target']

In [ ]:
predictions.to_csv('predictions.csv', sep=',', index=False)

Results from this very very simple model :  3344. Soit très mauvais modèle soit prb ID/target